# **Spotify Analysis: Fetching Lyrics from Genius API**


### Summary of the Lyrics Cleaning Process

This code aims to preprocess song lyrics by performing several cleaning operations to enhance the quality and readability of the text data.

1. **Fetching Lyrics**:
   - The `fetch_lyrics` function queries the Genius API to retrieve song lyrics based on the track name and artist. If the song is found, the lyrics are cleaned using the `clean_lyrics` function to ensure they are free from metadata and unnecessary characters.

2. **Initial Cleaning**:
   - The `clean_lyrics` function removes metadata (e.g., contributor information, translation notes) and non-ASCII characters from the lyrics, ensuring the text is clean and coherent.
   - The `clean_lyrics2` function further refines the lyrics by eliminating specific unwanted terms like "Translation," "Lyrics," and instances of "Embed" or "ArrayEmbed," as well as leading/trailing whitespace.

3. **Filtering**:
   - A regex pattern is used to identify and filter out rows containing unwanted formats or structures, such as artist features, ensuring only relevant lyrics are retained.

4. **Nonsense and Long Words Removal**:
   - The `remove_nonsense_and_sounds` function targets and removes nonsensical camel-case strings and excessively long words (10 or more characters) to eliminate noise from the lyrics.

5. **Final Output**:
   - The cleaned lyrics are stored in a DataFrame, providing a refined dataset suitable for further analysis or processing, ensuring clarity and focus on meaningful content.

Overall, these cleaning steps are crucial for preparing song lyrics for analysis, sentiment assessment, or machine learning applications, allowing for more accurate insights and results.


In [1]:
#Install all relevant packages: 
%run pip_installs.py
%run packages.py

Defaulting to user installation because normal site-packages is not writeable


/Users/varshis./Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/varshis./nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/varshis./nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/varshis./nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/varshis./nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /U

In [2]:
import os
import pandas as pd

def read_and_process_csv(file_path):
    df = pd.read_csv(file_path)
    # Convert the 'Added At' column to datetime with UTC
    if 'Added At' in df.columns:
        df['Added At'] = pd.to_datetime(df['Added At'], utc=True)
    # Convert the 'Release Date' column to datetime
    if 'Release Date' in df.columns:
        df['Release Date'] = pd.to_datetime(df['Release Date'], errors='coerce')
    return df

# Define the path to the specific CSV file
file_path = 'processed_csv/master_df.csv'

# Read and process the CSV file
df = read_and_process_csv(file_path)

# Remove duplicates based on 'Track Name' and 'Artist Name(s)', keeping the first instance
deduped_df = df.drop_duplicates(subset=['Track Name', 'Artist Name(s)'])

# Sort the dataframe by 'Added At' in ascending order
sorted_df = deduped_df.sort_values('Added At')

# Print information about the final dataframe
print(f"Final dataframe shape: {sorted_df.shape}")
print(f"Date range: from {sorted_df['Added At'].min()} to {sorted_df['Added At'].max()}")


Final dataframe shape: (8614, 25)
Date range: from 2016-04-25 23:31:24+00:00 to 2024-08-19 16:15:35+00:00


In [3]:
sorted_df.columns

Index(['Spotify ID', 'Artist IDs', 'Track Name', 'Album Name',
       'Artist Name(s)', 'Release Date', 'Duration (ms)', 'Popularity',
       'Added By', 'Added At', 'Genres', 'Danceability', 'Energy', 'Key',
       'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness',
       'Liveness', 'Valence', 'Tempo', 'Time Signature', 'Playlist', 'Season'],
      dtype='object')

### Using the Genius API to get lyrics and clean the lyrics to then feed into a sentiment analysis model

__________________

In [4]:
import pandas as pd
import time
from tqdm import tqdm  # Library to add progress bars to loops
import re  # Regular expression library for text manipulation

# Function to clean the lyrics by removing unwanted patterns
def clean_lyrics(lyrics):
    # Remove anything within square brackets (like [Chorus], [Verse])
    lyrics = re.sub(r'\[.*?\]', '', lyrics)
    # Remove phrases indicating contributors to the lyrics (case insensitive)
    lyrics = re.sub(r'\b\d+\s+contributor(?:s)?\b', '', lyrics, flags=re.IGNORECASE)
    # Remove the word 'translation'
    lyrics = re.sub(r'\btranslation\b', '', lyrics, flags=re.IGNORECASE)
    # Remove any non-ASCII characters (such as emojis or special symbols)
    lyrics = re.sub(r'[^\x00-\x7F]+', '', lyrics)
    # Remove empty lines from the lyrics
    lyrics = '\n'.join([line for line in lyrics.split('\n') if line.strip()])
    return lyrics.strip()  # Return the cleaned lyrics, stripped of extra spaces

# Function to fetch and clean lyrics for a given track and artist using the Genius API
def fetch_lyrics(genius, track, artist):
    try:
        song = genius.search_song(track, artist)  # Search for the song using the Genius API
        if song:
            return clean_lyrics(song.lyrics)  # Clean and return the lyrics if the song is found
    except:
        pass  # Ignore any errors silently (e.g., API errors or song not found)
    return None  # Return None if no valid lyrics are found

# Assume 'result_df' is your input DataFrame with track and artist data
result_df = sorted_df.copy()  # Create a copy of the DataFrame
result_df['Lyrics_Clean'] = None  # Add a new column to store cleaned lyrics

lyrics_cache = {}  # Create a cache to store lyrics for previously searched songs

# Loop through each row of the DataFrame
for index, row in tqdm(result_df.iterrows(), total=len(result_df)):
    track = row['Track Name']  # Extract the track name
    artist = row['Artist Name(s)']  # Extract the artist name
    
    # Check if the lyrics are already in the cache
    if (track, artist) in lyrics_cache:
        result_df.at[index, 'Lyrics_Clean'] = lyrics_cache[(track, artist)]  # Assign from cache
    else:
        # Fetch and clean the lyrics using the Genius API
        lyrics = fetch_lyrics(genius, track, artist)
        if lyrics:
            result_df.at[index, 'Lyrics_Clean'] = lyrics  # Store the cleaned lyrics in the DataFrame
            lyrics_cache[(track, artist)] = lyrics  # Save to the cache to avoid redundant API calls
        
        time.sleep(1)  # Delay to respect the API rate limit

# Remove rows where the 'Lyrics_Clean' column is still empty (i.e., no lyrics were found)
result_df = result_df.dropna(subset=['Lyrics_Clean'])

# Clean up the lyrics column further by removing newline and special paragraph symbols
result_df['Lyrics_Clean'] = result_df['Lyrics_Clean'].replace(
    to_replace=[r'\n', r'\r\n', r'¶'], value=' ', regex=True)

# Print the number of songs successfully processed with valid lyrics
print(f"Processed {len(result_df)} songs with valid lyrics.")

  0%|          | 0/8614 [00:00<?, ?it/s]

Searching for "Desi Girl" by Vishal-Shekhar,Shankar Mahadevan,Sunidhi Chauhan,Vishal Dadlani...
No results found for: 'Desi Girl Vishal-Shekhar,Shankar Mahadevan,Sunidhi Chauhan,Vishal Dadlani'


  0%|          | 1/8614 [00:01<4:06:22,  1.72s/it]

Searching for "Tum Tak" by A.R. Rahman,Javed Ali,KIRTI SAGATHIA,Pooja AV...
Done.


  0%|          | 2/8614 [00:03<4:43:11,  1.97s/it]

...

Searching for "Invitation (feat. Kodie Shane)" by Ashnikko,Kodie Shane...
Done.


100%|█████████▉| 8613/8614 [13:23:45<00:02,  2.46s/it]

Searching for "Strangers" by Ethel Cain...
Done.


100%|██████████| 8614/8614 [13:23:49<00:00,  5.60s/it]


Processed 7556 songs with valid lyrics.


In [6]:
import re  # Import regular expression library for text manipulation
import pandas as pd  # Import pandas for DataFrame manipulation

# Function to clean the lyrics by removing specific unwanted patterns
def clean_lyrics2(text):
    # Remove "[number] Contributors" from the beginning of the lyrics
    text = re.sub(r'^\d+ Contributors,?\s*', '', text)
    
    # Remove "Translation" or "Lyrics" from the beginning (within the first 30 characters)
    text = re.sub(r'^(.{0,30}?)(Translation|Lyrics),?\s*', r'\1', text)
    
    # Remove instances of "ArrayEmbed" and "Embed" from the lyrics
    text = text.replace("ArrayEmbed", "").replace("Embed", "")
    
    # Strip leading and trailing whitespace from the cleaned text
    return text.strip()

# Sample regex pattern to identify rows that might contain unwanted structures
regex_pattern = r"(?i)(- .*? \(|feat\.|&|\w+ \- \w+)"  # Matches patterns like "- Artist (feat. Artist)" or "Song - Artist"

# Create a copy of the DataFrame and clean the lyrics using the new cleaning function
result_df2 = result_df.copy()
result_df2['Lyrics_Clean'] = result_df2['Lyrics_Clean'].apply(clean_lyrics2)  # Apply the cleaning function

# Filter out rows where Lyrics_Clean matches the regex pattern (indicating unwanted formats)
subset_without_lists = result_df2[~result_df2['Lyrics_Clean'].str.contains(regex_pattern, regex=True)]

# Display the resulting subset that contains cleaned lyrics without unwanted patterns


/var/folders/p_/sxdpc_pj2cq36rvp4xwys4q00000gn/T/ipykernel_85070/3387136392.py:26: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  subset_without_lists = result_df2[~result_df2['Lyrics_Clean'].str.contains(regex_pattern, regex=True)]


,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,Playlist,Season,Lyrics_Clean
5856,1wnozUBSe7OMpmZiTPJC5j,"1mYsTxnqsietFxj1OgoGbG,4W91bbPB2CTSsHwt7eqNl7,...",Tum Tak,Raanjhanaa (Original Motion Picture Soundtrack),"A.R. Rahman,Javed Ali,KIRTI SAGATHIA,Pooja AV",2013-05-31,304466,67,spotify:user:vrshi,2016-04-25 23:31:56+00:00,...,0.05,0.68,0.00,0.07,0.57,114.99,4.00,54n5k4r1,Spring,"sRomanizationTum Tak Lyrics - , , ..."
5857,4ila6GeGBPGmJTGRoHOV5E,"0oOet2f43PA68X5RxKobEy,20w1cujlzVI3LA0dItmMg1",Nagada Sang Dhol,Ram-Leela (Original Motion Picture Soundtrack),"Shreya Ghoshal,Osman Mir",2013-10-01,273319,0,spotify:user:vrshi,2016-04-25 23:33:02+00:00,...,0.16,0.27,0.00,0.04,0.66,113.31,3.00,54n5k4r1,Spring,sEnglishRomanizationNagada Sang Dhol Lyrics ...
5858,5OMUXgfXsSukZ0zxelpC3b,"1mYsTxnqsietFxj1OgoGbG,2IP4wcE4hRbDi1zdV8Si6g,...",Raanjhanaa,Raanjhanaa (Original Motion Picture Soundtrack),"A.R. Rahman,Jaswinder Singh,Shiraz Uppal",2013-05-31,254653,67,spotify:user:vrshi,2016-04-25 23:33:17+00:00,...,0.04,0.11,0.00,0.09,0.78,112.94,4.00,54n5k4r1,Spring,"1 ContributorRaanjhanaa , - , , , ..."
5860,4J9n7thOdqi8gxRGNyQoVC,"7HCqGPJcQTyGJ2yqntbuyr,6k6dimE4a6OFnD0HWmkIUW",Gal Mitthi Mitthi,Aisha (Original Motion Picture Soundtrack),"Amit Trivedi,Tochi Raina",2010-07-05,255906,59,spotify:user:vrshi,2016-04-25 23:33:39+00:00,...,0.05,0.13,0.02,0.18,0.94,129.95,4.00,54n5k4r1,Spring,1 ContributorGal Mitthi Mitthi LyricsGal mitth...
5863,3DFTyZw8YVaQu9BeZUMLss,0z4gvV4rjIZ9wHck67ucSV,Chammak Challo,Chammak Challo - Single,Akon,2011-06-03,227462,0,spotify:user:vrshi,2016-04-25 23:35:19+00:00,...,0.03,0.12,0.00,0.23,0.90,133.95,4.00,54n5k4r1,Spring,sEnglishRomanizationChammak Challo Lyrics Girl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9061,0V8FYVlBFuXXTIvRnMbZyS,5YGY8feqx7naU7z4HrwZM6,The Climb,Hannah Montana The Movie,Miley Cyrus,2009-01-01,236066,64,spotify:user:vrshi,2024-08-19 16:10:16+00:00,...,0.03,0.08,0.00,0.10,0.33,161.07,4.00,hannah_and_varshis_house_of_pain,Summer,sThe Climb Lyrics I can almost see it That dre...
9064,6Pof690ci3aferHd9VVQwO,7Ez6lTtSMjMf2YSYpukP1I,What Happens Now?,What Happens Now?,Dasha,2024-02-16,153070,5,spotify:user:vrshi,2024-08-19 16:11:22+00:00,...,0.05,0.02,0.00,0.08,0.42,127.17,4.00,hannah_and_varshis_house_of_pain,Summer,"What Happens Now? Take me back to June, sky wa..."
9067,3BHFResGQiUvbYToUdaDQz,246dkjvS1zLTtiykXe5h60,Enough Is Enough,AUSTIN,Post Malone,2023-07-28,165175,68,spotify:user:vrshi,2024-08-19 16:12:30+00:00,...,0.03,0.01,0.00,0.11,0.38,166.00,4.00,hannah_and_varshis_house_of_pain,Summer,"sPortugusItalianoEnough Is Enough Lyrics 2 AM,..."
9069,46aopTu45teng25ROkoeXk,"3PyJHH2wyfQK3WZrk9rpmP,1CUeN4GnHAGUk9nAXPorF4",Invitation (feat. Kodie Shane),Unlikeable,"Ashnikko,Kodie Shane",2018-11-23,159140,56,spotify:user:vrshi,2024-08-19 16:12:46+00:00,...,0.15,0.03,0.00,0.14,0.82,128.01,4.00,hannah_and_varshis_house_of_pain,Summer,Invitation Fuck you mean you need it? He said ...


In [7]:
import re  # Import the regular expression library for text manipulation

# Define a function to remove nonsense strings and onomatopoeia
def remove_nonsense_and_sounds(text):
    # Remove known nonsense strings (patterns similar to given example)
    text = re.sub(r'\b[A-Z][a-z]+(?:[A-Z][a-z]+){2,}\b', '', text)  
    # Removes camel-case nonsense words (e.g., "CamelCaseExample") by matching words with 
    # at least two capitalized segments, ensuring they are surrounded by word boundaries.
    
    text = re.sub(r'\b\w{10,}\b', '', text)  
    # Removes very long words (10 or more characters) that may not be meaningful,
    # again ensuring these words are surrounded by word boundaries.

    return text  # Return the cleaned text

# Apply the function to the 'Lyrics_Clean' column in the DataFrame
subset_without_lists['Lyrics_Clean'] = subset_without_lists['Lyrics_Clean'].apply(remove_nonsense_and_sounds)


/var/folders/p_/sxdpc_pj2cq36rvp4xwys4q00000gn/T/ipykernel_85070/2280641936.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_without_lists['Lyrics_Clean'] = subset_without_lists['Lyrics_Clean'].apply(remove_nonsense_and_sounds)


In [8]:
import os
# Define the output directory
output_dir = 'processed_csv'

# Define the filenames for the CSV files
cleaned_df_filename = 'clean_data_all.csv'

# Create the full paths to the output CSV files
clean_track_path = os.path.join(output_dir, cleaned_df_filename)

# Save the DataFrames as CSV files
subset_without_lists.to_csv(clean_track_path, index=False)